In [ ]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.converters import pdf_converter
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model


/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
download_model(model='bert-squad_1.1', dir='./models')

In [ ]:

!ls models

bert_qa.joblib


In [ ]:
!mkdir docs

In [ ]:
!wget -P ./docs/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q3/AMZN-Q3-2020-Earnings-Release.pdf
!wget -P ./docs/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q2/Q2-2020-Amazon-Earnings-Release.pdf
!wget -P ./docs/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/Q1/AMZN-Q1-2020-Earnings-Release.pdf
!wget -P ./docs/ https://s2.q4cdn.com/299287126/files/doc_news/archive/Amazon-Q4-2019-Earnings-Release.pdf
!wget -P ./docs/ https://s2.q4cdn.com/299287126/files/doc_news/archive/Q3-2019-Amazon-Financial-Results.pdf

--2021-07-30 11:24:57--  https://s2.q4cdn.com/299287126/files/doc_financials/2020/q3/AMZN-Q3-2020-Earnings-Release.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.1, 68.70.205.4, 68.70.205.3, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 945573 (923K) [application/pdf]
Saving to: ‘./docs/AMZN-Q3-2020-Earnings-Release.pdf’

AMZN-Q3-2020-Earnin 100%[===================>] 923.41K  --.-KB/s    in 0.07s   

2021-07-30 11:24:57 (13.8 MB/s) - ‘./docs/AMZN-Q3-2020-Earnings-Release.pdf’ saved [945573/945573]

--2021-07-30 11:24:57--  https://s2.q4cdn.com/299287126/files/doc_financials/2020/q2/Q2-2020-Amazon-Earnings-Release.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.1, 68.70.205.4, 68.70.205.3, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 366951 (358K) [application/pdf]
Saving to: ‘./docs/Q2-2020-Amazon-Ea

In [ ]:
df = pdf_converter(directory_path='./docs/')

In [ ]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib', max_df=1.0)

cdqa_pipeline.fit_retriever(df=df)

100%|██████████| 231508/231508 [00:00<00:00, 2762017.67B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [ ]:
import joblib
joblib.dump(cdqa_pipeline, './models/bert_qa_custom.joblib')

['./models/bert_qa_custom.joblib']

In [ ]:

cdqa_pipeline=joblib.load('./models/bert_qa_custom.joblib')
cdqa_pipeline

QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [ ]:

query = 'How many jobs are created in 2020?'
prediction = cdqa_pipeline.predict(query)

In [ ]:
prediction

('1.1 million',
 'Q2-2020-Amazon-Earnings-Release',
 'Empowering Small and Medium-Sized Businesses• Amazon released its 2020 Small and Medium-Sized Business (SMB) Impact Report, highlighting how SMBs selling in its U.S. store have sold more than 3.4 billion products in the past year and created an estimated 1.1 million jobs.• In partnership with the British small business support network “Enterprise Nation,” Amazon launched the Amazon Small Business Accelerator, which aims to support more than 200,000 small businesses across the U.K. negatively impacted by the COVID-19 crisis. Amazon hosted a week-long boot camp in the U.K. to help 1,000 offline businesses get online, and offered free services, AWS credits, training, and support.• In the U.K., Amazon worked with the British Chambers of Commerce to give up to 1,000 businesses tours of Amazon fulfillment centers, helping other companies learn from the safety measures Amazon has put in place within its own operations so the businesses can

In [ ]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: How many jobs are created in 2020?
answer: 1.1 million
title: Q2-2020-Amazon-Earnings-Release
paragraph: Empowering Small and Medium-Sized Businesses• Amazon released its 2020 Small and Medium-Sized Business (SMB) Impact Report, highlighting how SMBs selling in its U.S. store have sold more than 3.4 billion products in the past year and created an estimated 1.1 million jobs.• In partnership with the British small business support network “Enterprise Nation,” Amazon launched the Amazon Small Business Accelerator, which aims to support more than 200,000 small businesses across the U.K. negatively impacted by the COVID-19 crisis. Amazon hosted a week-long boot camp in the U.K. to help 1,000 offline businesses get online, and offered free services, AWS credits, training, and support.• In the U.K., Amazon worked with the British Chambers of Commerce to give up to 1,000 businesses tours of Amazon fulfillment centers, helping other companies learn from the safety measures Amazon has pu

In [ ]:
query = 'How many full time employees are on Amazon roll?'
prediction = cdqa_pipeline.predict(query)

In [ ]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: How many full time employees are on Amazon roll?
answer: 650,000
title: Q2-2020-Amazon-Earnings-Release
paragraph: Supporting Employees• Amazon’s top priority is providing for the health and safety of our employees and partners, and the company spent more than $4 billion in the second quarter on incremental COVID-19 related initiatives to help keep employees safe, provide additional compensation to our employees and delivery partners, and deliver products to customers.• Amazon provided a one-time Thank You bonus totaling over $500 million to all front-line employees and partners who were with the company throughout the month of June.• Amazon introduced a new family backup care benefit through Care.com to 650,000 full and part-time Amazon and Whole Foods Market employees in the U.S. This benefit provides employees with up to 10 days of company-subsidized emergency backup child or adult care.• Amazon introduced Distance Assistant to help keep employees safe by providing them with 

In [ ]:
query = 'What is the impact of COVID on business?'
prediction = cdqa_pipeline.predict(query, n_predictions=5)

In [ ]:

print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: What is the impact of COVID on business?
answer: ('negatively impacted by the COVID-19 crisis', 'Q2-2020-Amazon-Earnings-Release', 'Empowering Small and Medium-Sized Businesses• Amazon released its 2020 Small and Medium-Sized Business (SMB) Impact Report, highlighting how SMBs selling in its U.S. store have sold more than 3.4 billion products in the past year and created an estimated 1.1 million jobs.• In partnership with the British small business support network “Enterprise Nation,” Amazon launched the Amazon Small Business Accelerator, which aims to support more than 200,000 small businesses across the U.K. negatively impacted by the COVID-19 crisis. Amazon hosted a week-long boot camp in the U.K. to help 1,000 offline businesses get online, and offered free services, AWS credits, training, and support.• In the U.K., Amazon worked with the British Chambers of Commerce to give up to 1,000 businesses tours of Amazon fulfillment centers, helping other companies learn from the sa